In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import json
from equistore import Labels, TensorBlock, TensorMap
from utils.builder import DescriptorBuilder
import ase.io
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal
from utils.hamiltonians import fix_pyscf_l1, dense_to_blocks, blocks_to_dense, couple_blocks, decouple_blocks
import matplotlib.pyplot as plt
from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
from utils.pair_features import *

## Manipulate Hamiltonian into blocks

In [3]:
frames = ase.io.read("data/water-hamiltonian/water_coords_1000.xyz",":1")
for f in frames:
    f.cell = [100,100,100]
    f.positions += 50

In [4]:
#jorbs = json.load(open('data/water-hamiltonian/water_orbs.json', "r"))
jorbs = json.loads(json.load(open('data/water-hamiltonian/water_orbs.json', "r")))
orbs = {}
zdic = {"O" : 8, "H":1}
for k in jorbs:
    orbs[zdic[k]] = jorbs[k]

In [5]:
hams = np.load("data/water-hamiltonian/water_saph_orthogonal.npy", allow_pickle=True)
for i, f in enumerate(frames):
    hams[i] = fix_pyscf_l1(hams[i], f, orbs)

blocks = dense_to_blocks(hams, frames, orbs)

In [6]:
cg = ClebschGordanReal(4)

In [7]:
coupled = couple_blocks(blocks, cg)

fock_bc = coupled

## Feature computation

In [8]:
rascal_hypers = {
    "interaction_cutoff": 3.5,
    "cutoff_smooth_width": 0.5,
    "max_radial": 3,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
}

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)

In [9]:
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

In [10]:
rho0ij = rho0ij_builder(rascal_hypers, frames)

In [11]:
total_species = sorted(set(rhoi.keys['center_species']))
# total_species = list(np.sort(np.asarray(total_species)))
lmax=rascal_hypers["max_angular"]
nmax=rascal_hypers["max_radial"]

In [12]:
blocks = []
for l in range(lmax+1):
    for sp_i in total_species:
        for sp_k in total_species:
            n_selected = nmax#len(np.where(opt_eva[l] > sel_thresh)[0])    
            de_block = rhoi.block(center_species = sp_i, neighbor_species=sp_k, spherical_harmonics_l = l)
            block = TensorBlock(
                values = de_block.values,
                samples = de_block.samples,
                components = [Labels(["m"],np.asarray(range(-l,l+1), dtype=np.int32).reshape(-1,1))],
                properties = Labels(["n"], np.asarray([[n] for n in range(nmax)], dtype=np.int32))
            )
            
            blocks.append( block )

acdc_nu1 = TensorMap(
    keys = Labels(names=["L", "nu", "sigma","species_i", "neighbor_species"], 
                        values=np.asarray([[ l, 1, 1, sp_i, sp_k] for l in range(rascal_hypers["max_angular"]+1) 
                                                        for sp_i in total_species
                                                        for sp_k in total_species], dtype=np.int32)
                                     ), 
                      blocks = blocks
                     )
#move neighbor species to features  
# acdc_nu1.sparse_to_features('neighbor_species')

In [13]:
rho1 = acdc_nu1.keys_to_properties("neighbor_species")

In [14]:
rho1ij=tensor_g_rho_nu(rho0ij, acdc_nu1, rascal_hypers, cg)

(0, 0, 1, 1, 1, 1) done
(0, 1, 1, 1, 1, 1) done
(0, 1, 1, -1, 1, 1) done
(0, 2, 1, 1, 1, 1) done
(0, 2, 1, -1, 1, 1) done
(1, 0, 1, 1, 1, 1) done
(-1, 0, 1, 1, 1, 1) done
(1, 1, 1, 1, 1, 1) done
(1, 1, 1, -1, 1, 1) done
(-1, 1, 1, 1, 1, 1) done
(-1, 1, 1, -1, 1, 1) done
(1, 2, 1, 1, 1, 1) done
(1, 2, 1, -1, 1, 1) done
(-1, 2, 1, 1, 1, 1) done
(-1, 2, 1, -1, 1, 1) done
(2, 0, 1, 1, 1, 8) done
(2, 1, 1, 1, 1, 8) done
(2, 1, 1, -1, 1, 8) done
(2, 2, 1, 1, 1, 8) done
(2, 2, 1, -1, 1, 8) done
(0, 0, 1, 1, 8, 8) done
(0, 1, 1, 1, 8, 8) done
(0, 1, 1, -1, 8, 8) done
(0, 2, 1, 1, 8, 8) done
(0, 2, 1, -1, 8, 8) done


/Users/divya/scratch/aml-hamiltonian-1234/utils/pair_features.py:142: FutureWarning: elementwise == comparison failed and returning scalar instead; this will raise an error or perform elementwise comparison in the future.
  if i not in new_sparse_labels:


In [15]:
feats = rho1ij

## Testing model on one structure

In [16]:
from utils.Hamiltonian_model_utils import *

In [17]:
FR = Fock_regression(jorbs)

In [18]:
FR.fit(feats, fock_bc)

In [19]:
pred = FR.predict(feats)

(0, 8, 2, 0, 8, 2, 0, 0) (1, 1, 1)
(0, 8, 2, 0, 8, 2, 1, 1) (1, 3, 1)
(0, 8, 2, 1, 8, 2, 1, 0) (1, 1, 1)
(0, 8, 2, 1, 8, 2, 1, 2) (1, 5, 1)
(2, 1, 1, 0, 8, 2, 0, 0) (2, 1, 1)
(2, 1, 1, 0, 8, 2, 1, 1) (2, 3, 1)
(0, 1, 1, 0, 1, 1, 0, 0) (2, 1, 1)
(1, 1, 1, 0, 1, 1, 0, 0) (1, 1, 1)


In [20]:
for i in range(len(fock_bc.keys)):
    print(fock_bc.block(i).values -pred.block(i).values )

[[[-0.83320298]]]
[[[-8.68039665e-10]
  [ 1.27389862e-01]
  [ 9.41524938e-02]]]
[[[0.64651612]]]
[[[ 3.94035545e-11]
  [ 1.15190702e-15]
  [ 3.82237770e-02]
  [-1.13063712e-01]
  [ 1.71484860e-01]]]
[[[-0.50162485]]

 [[-0.39546113]]]
[[[ 5.15115859e-10]
  [-4.17477906e-01]
  [ 8.02670810e-02]]

 [[-1.12716623e-09]
  [ 4.78505673e-02]
  [-4.06783377e-01]]]
[[[-0.43368077]]

 [[-0.36748818]]]
[[[-0.27498094]]]
